In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important;margin:auto; }</style>"))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import sklearn

In [3]:
#Reading train data
train = pd.read_csv('TrainingData_V1.csv',index_col=False) 

In [4]:
#holding out 5% of data for unseen data
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,0:-1], train.iloc[:,-1], test_size=0.05, random_state=42,shuffle = True)

In [5]:
# train = pd.concat([X_train,y_train],axis = 1)
# cv = pd.concat([X_test,y_test],axis = 1)

In [6]:
train.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,22-06-2016,27-06-2016,643,38,navy,30,49.9,30822,Mrs,17-04-1969,1013,23-06-2016,0
1,10,22-06-2016,27-06-2016,195,xxl,grey,46,19.9,30823,Mrs,22-04-1970,1001,15-03-2015,1
2,11,22-06-2016,05-07-2016,25,xxl,grey,5,79.9,30823,Mrs,22-04-1970,1001,15-03-2015,0
3,32,23-06-2016,26-06-2016,173,m,brown,20,19.9,17234,Mrs,09-01-1960,1013,17-02-2015,0
4,43,23-06-2016,26-06-2016,394,40,black,44,90.0,30827,Mrs,NaN,1006,09-02-2016,1


In [7]:
#cv.tail()

## Feature creation and transformation

#### Converting dates columns into date format

In [8]:
date_cols = ['order_date','delivery_date','user_dob','user_reg_date']
for col in date_cols:
    train[col] = pd.to_datetime(train[col],format="%d-%m-%Y")

In [9]:
train.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,1969-04-17,1013,2016-06-23,0
1,10,2016-06-22,2016-06-27,195,xxl,grey,46,19.9,30823,Mrs,1970-04-22,1001,2015-03-15,1
2,11,2016-06-22,2016-07-05,25,xxl,grey,5,79.9,30823,Mrs,1970-04-22,1001,2015-03-15,0
3,32,2016-06-23,2016-06-26,173,m,brown,20,19.9,17234,Mrs,1960-01-09,1013,2015-02-17,0
4,43,2016-06-23,2016-06-26,394,40,black,44,90.0,30827,Mrs,NaT,1006,2016-02-09,1


#### Creating new columns in days

In [10]:

train['total_delivery_time'] = (train['delivery_date'] - train['order_date']).dt.days
train['user_age'] = round(((train['order_date']- train['user_dob']).dt.days)/365,0)
train['user_loyalty_in_days'] = (train['order_date'] - train['user_reg_date']).dt.days
# train = train.drop(['order_item_id','delivery_date','order_date','user_dob','user_reg_date',
#                     'item_id','item_size','item_color','brand_id','user_id','user_state'],axis =1)
# train = train.drop(['order_item_id','delivery_date','order_date','user_dob','user_reg_date',
#                     'item_id','item_size','item_color','user_id'],axis =1)

#### User ID return rate

In [11]:
user_return = train.groupby(['user_id']).agg(returns = ('return' ,'sum'))
user_total_orders = train.groupby(['user_id']).agg(total_order = ('return' ,'count'))
user_return_behav = pd.merge(user_return,user_total_orders,'inner',on = 'user_id')
user_return_behav['user_return_rate'] = user_return_behav['returns']/user_return_behav['total_order'] 
#user_return_behav = user_return_behav[['user_id','user_return_perc']]
user_return_behav = user_return_behav.reset_index(level =0)[['user_id','user_return_rate']]

In [12]:
user_return_behav.head()

,user_id,user_return_rate
0,9,0.800000
1,11,0.000000
2,30,0.500000
3,40,0.666667
4,48,0.750000


#### Item return rate

In [13]:
item_return = train.groupby(['item_id']).agg(returns = ('return' ,'sum'))
item_total_orders = train.groupby(['item_id']).agg(total_order = ('return' ,'count'))
item_return_behav = pd.merge(item_return,item_total_orders,'inner',on = 'item_id')
item_return_behav['item_return_rate'] = item_return_behav['returns']/item_return_behav['total_order'] 
#user_return_behav = user_return_behav[['user_id','user_return_perc']]
item_return_behav = item_return_behav.reset_index(level =0)[['item_id','item_return_rate']]

In [14]:
item_return_behav.head()

,item_id,item_return_rate
0,1,0.450000
1,2,0.396552
2,3,0.323077
3,4,0.577982
4,5,0.476395


#### Brand return rate

In [15]:
brand_return = train.groupby(['brand_id']).agg(returns = ('return' ,'sum'))
brand_total_orders = train.groupby(['brand_id']).agg(total_order = ('return' ,'count'))
brand_return_behav = pd.merge(brand_return,brand_total_orders,'inner',on = 'brand_id')
brand_return_behav['brand_return_rate'] = brand_return_behav['returns']/brand_return_behav['total_order'] 
#user_return_behav = user_return_behav[['user_id','user_return_perc']]
brand_return_behav = brand_return_behav.reset_index(level =0)[['brand_id','brand_return_rate']]

In [16]:
brand_return_behav.head()

,brand_id,brand_return_rate
0,1,0.491299
1,2,0.405045
2,3,0.366162
3,4,0.485175
4,5,0.386858


#### Title return rate

In [17]:
title_return = train.groupby(['user_title']).agg(returns = ('return' ,'sum'))
title_total_orders = train.groupby(['user_title']).agg(total_order = ('return' ,'count'))
title_return_behav = pd.merge(title_return,title_total_orders,'inner',on = 'user_title')
title_return_behav['title_return_rate'] = title_return_behav['returns']/title_return_behav['total_order'] 
#user_return_behav = user_return_behav[['user_id','user_return_perc']]
title_return_behav = title_return_behav.reset_index(level =0)[['user_title','title_return_rate']]

In [18]:
title_return_behav.head()

,user_title,title_return_rate
0,Company,0.609524
1,Family,0.392330
2,Mr,0.422477
3,Mrs,0.460317
4,not reported,0.333333


In [21]:
title_return_behav = title_return_behav.iloc[:4,]

In [22]:
title_return_behav.head()

,user_title,title_return_rate
0,Company,0.609524
1,Family,0.392330
2,Mr,0.422477
3,Mrs,0.460317


#### Size return rate

In [23]:
# size_return = train.groupby(['item_size']).agg(returns = ('return' ,'sum'))
# size_total_orders = train.groupby(['item_size']).agg(total_order = ('return' ,'count'))
# size_return_behav = pd.merge(size_return,size_total_orders,'inner',on = 'item_size')
# size_return_behav['size_return_rate'] = size_return_behav['returns']/size_return_behav['total_order'] 
# #user_return_behav = user_return_behav[['user_id','user_return_perc']]
# size_return_behav = size_return_behav.reset_index(level =0)[['item_size','size_return_rate']]

In [24]:
#size_return_behav.head()

#### Colour return rate

In [25]:
# color_return = train.groupby(['item_color']).agg(returns = ('return' ,'sum'))
# color_total_orders = train.groupby(['item_color']).agg(total_order = ('return' ,'count'))
# color_return_behav = pd.merge(color_return,color_total_orders,'inner',on = 'item_color')
# color_return_behav['color_return_rate'] = color_return_behav['returns']/color_return_behav['total_order'] 
# #user_return_behav = user_return_behav[['user_id','user_return_perc']]
# color_return_behav = color_return_behav.reset_index(level =0)[['item_color','color_return_rate']]
# color_return_behav

In [26]:
# color_return_behav.head()

In [27]:
# color_return_behav = color_return_behav.iloc[1:,:]

In [28]:
# color_return_behav.head()

#### Joining it with train dataset

In [29]:
train = pd.merge(train,user_return_behav,on ='user_id',how ='left')
train = pd.merge(train,item_return_behav,on ='item_id',how = 'left')
train = pd.merge(train,brand_return_behav,on ='brand_id',how ='left')
train = pd.merge(train,title_return_behav,on ='user_title',how ='left')

# train = pd.merge(train,size_return_behav,on ='item_size',how ='left')
# train = pd.merge(train,color_return_behav,on ='item_color',how ='left')
#train[['user_return_rate','item_return_rate','brand_return_rate','size_return_rate','color_return_rate']] = train[['user_return_rate','item_return_rate','brand_return_rate','size_return_rate','color_return_rate']].fillna(0)
train[['user_return_rate','item_return_rate','brand_return_rate','user_title']] = train[['user_return_rate','item_return_rate','brand_return_rate','user_title']].fillna(0)

In [30]:
train.head(5)

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_state,user_reg_date,return,total_delivery_time,user_age,user_loyalty_in_days,user_return_rate,item_return_rate,brand_return_rate,title_return_rate
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,...,1013,2016-06-23,0,5.0,47.0,-1,0.000000,0.222222,0.351724,0.460317
1,10,2016-06-22,2016-06-27,195,xxl,grey,46,19.9,30823,Mrs,...,1001,2015-03-15,1,5.0,46.0,465,0.588235,0.345238,0.482962,0.460317
2,11,2016-06-22,2016-07-05,25,xxl,grey,5,79.9,30823,Mrs,...,1001,2015-03-15,0,13.0,46.0,465,0.588235,0.407407,0.386858,0.460317
3,32,2016-06-23,2016-06-26,173,m,brown,20,19.9,17234,Mrs,...,1013,2015-02-17,0,3.0,56.0,492,0.428571,0.688406,0.588157,0.460317
4,43,2016-06-23,2016-06-26,394,40,black,44,90.0,30827,Mrs,...,1006,2016-02-09,1,3.0,NaN,135,1.000000,0.551724,0.536842,0.460317


#### Fixing column's distribution

In [31]:
def fix_column(x,mean,upper_limit):
    if x < 0:
        x = mean
    elif x > upper_limit:
        x = upper_limit
    else:
        pass
    return x

def column_metrics(col):
    mean = train[train[col]>0][col].mean()
    upper_limt = train[train[col]>0][col].quantile(0.95)
    return mean,upper_limt

columns = ['total_delivery_time','user_age','user_loyalty_in_days','item_price']

for col in columns:
    mean,upper_limit = column_metrics(col)
    train[col] = train[col].apply(lambda x : fix_column(x,mean,upper_limit))
    train[col] = train[col].fillna(mean)

In [32]:
train.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_state,user_reg_date,return,total_delivery_time,user_age,user_loyalty_in_days,user_return_rate,item_return_rate,brand_return_rate,title_return_rate
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,...,1013,2016-06-23,0,5.0,47.000000,344.776169,0.000000,0.222222,0.351724,0.460317
1,10,2016-06-22,2016-06-27,195,xxl,grey,46,19.9,30823,Mrs,...,1001,2015-03-15,1,5.0,46.000000,465.000000,0.588235,0.345238,0.482962,0.460317
2,11,2016-06-22,2016-07-05,25,xxl,grey,5,79.9,30823,Mrs,...,1001,2015-03-15,0,13.0,46.000000,465.000000,0.588235,0.407407,0.386858,0.460317
3,32,2016-06-23,2016-06-26,173,m,brown,20,19.9,17234,Mrs,...,1013,2015-02-17,0,3.0,56.000000,492.000000,0.428571,0.688406,0.588157,0.460317
4,43,2016-06-23,2016-06-26,394,40,black,44,90.0,30827,Mrs,...,1006,2016-02-09,1,3.0,52.330076,135.000000,1.000000,0.551724,0.536842,0.460317


#### Dropping unnecessary columns

In [33]:
train = train.drop(['order_item_id','delivery_date','order_date','user_dob','user_reg_date',
                    'item_id','item_size','item_color','brand_id','user_id','user_state','user_title'],axis =1)

In [34]:
train.head()

,item_price,return,total_delivery_time,user_age,user_loyalty_in_days,user_return_rate,item_return_rate,brand_return_rate,title_return_rate
0,49.9,0,5.0,47.000000,344.776169,0.000000,0.222222,0.351724,0.460317
1,19.9,1,5.0,46.000000,465.000000,0.588235,0.345238,0.482962,0.460317
2,79.9,0,13.0,46.000000,465.000000,0.588235,0.407407,0.386858,0.460317
3,19.9,0,3.0,56.000000,492.000000,0.428571,0.688406,0.588157,0.460317
4,90.0,1,3.0,52.330076,135.000000,1.000000,0.551724,0.536842,0.460317


## Modelling

In [40]:
import pycaret

In [41]:
from pycaret.classification import *
#from imblearn.combine import SMOTETomek
#from imblearn.under_sampling import TomekLinks
clf = setup(data = train,
                    #categorical_features=['user_title'],
                   target = 'return',
                   train_size = 0.95,
                   session_id=12345,
                  data_split_stratify = True,
                    categorical_imputation = 'mode',
                    normalize = True,
                    normalize_method='minmax',
                    transformation = True,
                    remove_outliers = True,
                    remove_multicollinearity = True,
                    feature_interaction = True
            
           )

,Description,Value
0,session_id,12345
1,Target,return
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(79945, 9)"
5,Missing Values,True
6,Numeric Features,8
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [42]:
compare_models(sort = "AUC")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7930,0.8886,0.8167,0.7561,0.7852,0.5860,0.5876,0.4450
gbc,Gradient Boosting Classifier,0.7930,0.8883,0.8142,0.7574,0.7848,0.5859,0.5874,2.1830
ada,Ada Boost Classifier,0.7902,0.8866,0.7899,0.7651,0.7773,0.5791,0.5795,0.6090
catboost,CatBoost Classifier,0.7893,0.8858,0.8091,0.7542,0.7807,0.5785,0.5798,19.4630
lr,Logistic Regression,0.7753,0.8700,0.7609,0.7559,0.7584,0.5484,0.5484,0.6950
lda,Linear Discriminant Analysis,0.7751,0.8698,0.7719,0.7501,0.7608,0.5487,0.5489,0.0580
qda,Quadratic Discriminant Analysis,0.7727,0.8658,0.8061,0.7311,0.7667,0.5461,0.5487,0.0370
nb,Naive Bayes,0.7690,0.8592,0.7733,0.7401,0.7563,0.5370,0.5375,0.0310
rf,Random Forest Classifier,0.7626,0.8587,0.7624,0.7352,0.7485,0.5239,0.5242,2.2870
knn,K Neighbors Classifier,0.7522,0.8333,0.7486,0.7254,0.7368,0.5027,0.5030,0.6920


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
model = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7904,0.8887,0.7808,0.7702,0.7755,0.5790,0.5791
1,0.7932,0.8876,0.7955,0.7670,0.7810,0.5852,0.5856
2,0.7942,0.8870,0.7931,0.7698,0.7813,0.5870,0.5872
3,0.7861,0.8848,0.7826,0.7623,0.7723,0.5708,0.5709
4,0.7872,0.8875,0.7955,0.7576,0.7761,0.5737,0.5743
5,0.7917,0.8858,0.8050,0.7598,0.7818,0.5828,0.5837
6,0.7891,0.8855,0.7856,0.7654,0.7754,0.5766,0.5768
7,0.7874,0.8856,0.7781,0.7666,0.7723,0.5729,0.5730
8,0.7932,0.8902,0.8026,0.7633,0.7824,0.5856,0.5863
9,0.7897,0.8834,0.7807,0.7690,0.7748,0.5776,0.5777


In [44]:
#final_model = finalize_model('model')

In [45]:
tuned_model = tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7809,0.8770,0.7584,0.7664,0.7624,0.5591,0.5591
1,0.7839,0.8772,0.7760,0.7621,0.7690,0.5661,0.5661
2,0.7770,0.8734,0.7691,0.7545,0.7617,0.5522,0.5523
3,0.7785,0.8727,0.7700,0.7565,0.7632,0.5552,0.5553
4,0.7849,0.8773,0.7799,0.7617,0.7707,0.5682,0.5683
5,0.7781,0.8711,0.7638,0.7590,0.7614,0.5540,0.5540
6,0.7807,0.8742,0.7656,0.7624,0.7640,0.5592,0.5592
7,0.7708,0.8678,0.7706,0.7439,0.7571,0.5402,0.5405
8,0.7879,0.8796,0.7727,0.7704,0.7715,0.5737,0.5737
9,0.7745,0.8717,0.7535,0.7583,0.7559,0.5463,0.5463


In [46]:
#ensembled_model = ensemble_model(model,method='Boosting')

In [47]:
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [48]:
#interpret_model(ensembled_model, plot = 'correlation')

In [49]:
holdout_pred = predict_model(tuned_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ada Boost Classifier,0.7904,0.8838,0.7949,0.7592,0.7767,0.5794,0.5799


#### Trying non Linear Model

In [50]:
# from sklearn import svm
# # fit the model
# clf = svm.SVC(kernel='poly')
# clf.fit(train.drop(['return'],axis =1), train['return'])

In [51]:
# train_pred = clf.predict(train.drop(['return'],axis =1))

In [52]:
# train_pred

In [53]:
# from sklearn.metrics import classification_report
# print(classification_report(train_pred, train['return'].values))

## Prediction

#### Data Read

In [54]:
#test = cv
test = pd.read_excel('TestingData_For_Candidate.xlsx')

In [55]:
test.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date
0,26,2016-06-23,2016-06-26,92,xl,turquoise,42,69.90,9392,Mrs,1962-04-14,1010,2016-03-24
1,28,2016-06-23,2016-07-02,2,xxl,green,2,19.90,9392,Mrs,1962-04-14,1010,2016-03-24
2,31,2016-06-23,2016-06-26,32,m,brown,3,12.90,17234,Mrs,1960-01-09,1013,2015-02-17
3,37,2016-06-23,2016-06-26,895,38,white,39,39.95,30826,Mrs,1964-04-27,1001,2015-02-17
4,56,2016-06-23,NaT,5,l,white,5,69.90,30828,Mrs,1966-05-13,1008,2016-01-21


#### Applying same feature enginnering to test data set

In [56]:
#### Converting dates columns into date format
date_cols = ['order_date','delivery_date','user_dob','user_reg_date']
for col in date_cols:
    test[col] = pd.to_datetime(test[col],format="%d-%m-%Y")

#### Creating new columns in days
test['total_delivery_time'] = (test['delivery_date'] - test['order_date']).dt.days
test['user_age'] = round(((test['order_date']- test['user_dob']).dt.days)/365,0)
test['user_loyalty_in_days'] = (test['order_date'] - test['user_reg_date']).dt.days


#### Joining it with return rates
test = pd.merge(test,user_return_behav,on ='user_id',how ='left')
test = pd.merge(test,item_return_behav,on ='item_id',how = 'left')
test = pd.merge(test,brand_return_behav,on ='brand_id',how ='left')
test = pd.merge(test,title_return_behav,on ='user_title',how ='left')
# test = pd.merge(test,size_return_behav,on ='item_size',how ='left')
# test = pd.merge(test,color_return_behav,on ='item_color',how ='left')
#test[['user_return_rate','item_return_rate','brand_return_rate','size_return_rate','color_return_rate']] = test[['user_return_rate','item_return_rate','brand_return_rate','size_return_rate','color_return_rate']].fillna(0)
test[['user_return_rate','item_return_rate','brand_return_rate','title_return_rate']] = test[['user_return_rate','item_return_rate','brand_return_rate','title_return_rate']].fillna(0)

#### Fixing column's distribution

def fix_column_test(x,mean,upper_limit):
    if x < 0:
        x = mean
    elif x > upper_limit:
        x = upper_limit
    else:
        pass
    return x

def column_metrics_test(col):
    mean = test[test[col]>0][col].mean()
    upper_limt = test[test[col]>0][col].quantile(0.95)
    return mean,upper_limt

columns = ['total_delivery_time','user_age','user_loyalty_in_days','item_price']

for col in columns:
    mean,upper_limit = column_metrics_test(col)
    test[col] = test[col].apply(lambda x : fix_column_test(x,mean,upper_limit))
    test[col] = test[col].fillna(mean)
    
#### Dropping unnecessary columns   
test = test.drop(['order_item_id','delivery_date','order_date','user_dob','user_reg_date',
                    'item_id','item_size','item_color','brand_id','user_id','user_state','user_title'],axis =1)    
    

In [57]:
test.head()

,item_price,total_delivery_time,user_age,user_loyalty_in_days,user_return_rate,item_return_rate,brand_return_rate,title_return_rate
0,69.90,3.000000,54.0,91.0,0.863636,0.245902,0.444661,0.460317
1,19.90,9.000000,54.0,91.0,0.863636,0.396552,0.405045,0.460317
2,12.90,3.000000,56.0,492.0,0.428571,0.279612,0.366162,0.460317
3,39.95,3.000000,52.0,492.0,1.000000,0.540323,0.501603,0.460317
4,69.90,7.515656,50.0,154.0,0.921053,0.476395,0.386858,0.460317


In [58]:
test.describe()

,item_price,total_delivery_time,user_age,user_loyalty_in_days,user_return_rate,item_return_rate,brand_return_rate,title_return_rate
count,20055.000000,20055.000000,20055.000000,20055.000000,20055.000000,20055.000000,20055.000000,20055.000000
mean,62.638180,6.431663,51.674765,344.005303,0.441350,0.456859,0.458901,0.458292
std,40.771658,8.313409,8.124075,174.476606,0.327625,0.165624,0.095705,0.017773
min,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.900000,2.000000,47.000000,195.000000,0.142857,0.352941,0.386858,0.460317
50%,49.900000,3.000000,52.442055,345.329183,0.466667,0.470790,0.470726,0.460317
75%,79.900000,7.515656,57.000000,503.000000,0.666667,0.569231,0.509158,0.460317
max,159.900000,35.000000,67.000000,559.000000,1.000000,1.000000,1.000000,0.609524


#### Prediction

In [60]:
#predcition = predict_model(tuned_model,data = test.drop(['return'],axis =1))
predcition = predict_model(tuned_model,data = test)
#predictions = predict_model(rf, data = diabetes)

In [61]:
predcition.head()

,item_price,total_delivery_time,user_age,user_loyalty_in_days,user_return_rate,item_return_rate,brand_return_rate,title_return_rate,Label,Score
0,69.90,3.000000,54.0,91.0,0.863636,0.245902,0.444661,0.460317,1,0.5285
1,19.90,9.000000,54.0,91.0,0.863636,0.396552,0.405045,0.460317,1,0.5262
2,12.90,3.000000,56.0,492.0,0.428571,0.279612,0.366162,0.460317,0,0.5285
3,39.95,3.000000,52.0,492.0,1.000000,0.540323,0.501603,0.460317,1,0.6739
4,69.90,7.515656,50.0,154.0,0.921053,0.476395,0.386858,0.460317,1,0.5407


#### Evaluation

In [63]:
#compare = pd.concat([predcition,test],axis =1)

In [66]:
#compare.head()

In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(compare['Label'].values, compare['return'].values))

In [65]:
predcition.to_csv('testset_results.csv')

In [ ]:
#compare.to_csv('testset_results.csv')